This is to import all the modules used

In [1]:

import csv as csv
import numpy as np
import pandas
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold




This is to import both the test and train data

In [2]:

#load in csv file train data
data = pandas.read_csv('train.csv', header = 0)

#load in csv file test data
data_test = pandas.read_csv('test.csv', header = 0)


Exploratory analysis on train data - not to be used when running the model

In [3]:
data.describe()

/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Feature Engineering Part 1:
Change some of the string variables to numerical variables. 

In [4]:
#change the Sex variable to Gender
data['Gender']=0
data['Gender'] = data['Sex'].map( {'female':1, 'male':0}).astype(int)

#title



data['Title']=data['Name'].str.split(',').str.get(1).str.split().str.get(0)
data.loc[(data['Title']=='Mlle.') ,'Title']="Miss."
data.loc[(data['Title']=="Mme.") ,'Title']="Mrs."
data.loc[(data['Title']=="Ms.") ,'Title']="Mrs."

data['Rare']=data['Title'].isin(["Master.", "Miss.", "Mrs.", "Mr."])

data.loc[(data['Rare']==False),'Title']="Other"
data = data.drop(['Rare'], axis = 1)

data['Mother']=0
data.loc[((data['Title']=="Mrs.")&(data['Parch']>0)),'Mother']=1

#Family Size
data['FamilySize']=data['SibSp']+data['Parch']+1

#Fare
data['Fare']=data['Fare']/data['FamilySize']
data['Fare2']=2
data.loc[(data['Fare']<10),'Fare2']=1
data.loc[(data['Fare']>40),'Fare2']=3



#Fill in missing ages with the Median and call AgeFill
data['AgeFill']=data['Age']
median_ages = np.zeros((2,3))

#fill in median of master for those with missing values 
data.loc[(data['Title']=='Master')& (data.Age.isnull()),'AgeFill']=data[data['Title']=='Master']['Age'].dropna().median()

for i in range(0,2):
	for j in range(0,3):
		median_ages[i,j] = data[(data['Gender'] == i ) & (data['Pclass'] ==j+1)]['Age'].dropna().median()


for i in range(0,2):
	for j in range(0,3):
		data.loc[ (data.Age.isnull() & (data['Gender']==i) & (data['Pclass']==j+1)), 'AgeFill']=median_ages[i,j]

        


#Normalize data for fare
from sklearn.preprocessing import scale
data['Fare']=scale(data['Fare'])        

#Embarked
data['Embarked']=data['Embarked'].fillna("S")
data['Embarked'][data['Embarked']=="S"]=0
data['Embarked'][data['Embarked']=="C"]=1
data['Embarked'][data['Embarked']=="Q"]=2
#data['Embarked']=data['Embarked'].astype(int)


#Ticket
#data['Tix']=data['Ticket'].str.split().str.get(0)



#print data['Ticket'].str.split().head(10)



print data.info()

data.describe()


/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
Gender         891 non-null int64
Title          891 non-null object
Mother         891 non-null int64
FamilySize     891 non-null int64
Fare2          891 non-null int64
AgeFill        891 non-null float64
dtypes: float64(3), int64(9), object(6)
memory usage: 125.4+ KB
None


/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender,Mother,FamilySize,Fare2,AgeFill
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,8.910000e+02,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,-2.990500e-17,0.352413,0.062851,1.904602,1.527497,29.112424
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,1.000562e+00,0.477990,0.242831,1.613459,0.672139,13.304424
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,-5.559950e-01,0.000000,0.000000,1.000000,1.000000,0.420000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,-3.536006e-01,0.000000,0.000000,1.000000,1.000000,21.500000
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,-3.242883e-01,0.000000,0.000000,1.000000,1.000000,26.000000
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,1.046949e-01,1.000000,0.000000,2.000000,2.000000,36.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,1.374643e+01,1.000000,1.000000,11.000000,3.000000,80.000000


In [5]:
#create new variables for family name and create a variable that predicts whether the family survived or not 
#clued in as 1 if anyone from the family died


#create a Last Name column of data
#data['LastName']=data['Name'].str.split(',').str.get(0)



####if "master, then guess young age for boy


#data['Title']=le_name.fit_transform(data['Title'])

#pandas.crosstab(data['Survived'], data['Title']).transpose()

#classify survival by family type
#pandas.crosstab(data['Survived'], data['LastName']).transpose()

#family_died = np.zeros((2,len(data['LastName'])))

#for i in range(0,len(data['LastName'])):
 #   family_died[0,i]=data[(data['LastName']==i)]['Survived'].sum()

#for i in range(0,len(data['LastName'])):
 #   if family_died[0,i]==0:
  #      family_died[1,i]=1

#print family_died.transpose()[10]
#pandas.crosstab(data['Survived'], data['LastName']).transpose()

#data['FamilyDied']=0
#for i in range(0,len(data['LastName'])):
 #   data.loc[(data['LastName']==i),'FamilyDied']=family_died[1,i]

#data.info()

Is survival linked to port of embarkment or class?

Survival was worst for embarked S and for class 3. These were most likely the poor individuals. The women who perished seem to have come from the 'S' embarkment. 

Is survival linked to family size?

It looks like men were travelling alone or those with larger families both perished more. Women who were travelling alone tended to survive more. 

Feature Engineering for the test data set.

In [6]:

data_test['Gender']=4
data_test['Gender'] = data_test['Sex'].map( {'female':1, 'male':0}).astype(int)
data_test['AgeFill']=data_test['Age']

##title

data_test['Title']=data_test['Name'].str.split(',').str.get(1).str.split().str.get(0)
data_test.loc[(data_test['Title']=='Mlle.') ,'Title']="Miss."
data_test.loc[(data_test['Title']=="Mme.") ,'Title']="Mrs."
data_test.loc[(data_test['Title']=="Ms.") ,'Title']="Mrs."

data_test['Rare']=data_test['Title'].isin(["Master.", "Miss.", "Mrs.", "Mr."])

data_test.loc[(data_test['Rare']==False),'Title']="Other"

#mother variable
data_test['Mother']=0
data_test.loc[((data_test['Title']=="Mrs.")&(data_test['Parch']>0)),'Mother']=1

data_test = data_test.drop(['Rare'], axis = 1)
#data_test['Title']=le_name.fit_transform(data_test['Title'])

#fill in median of master for those with missing values 
data_test.loc[(data_test['Title']=='Master')& (data_test.Age.isnull()),'AgeFill']=data_test[data_test['Title']=='Master']['Age'].dropna().median()



#Family Size
data_test['FamilySize']=data_test['SibSp']+data['Parch']+1


#Fare
data_test['Fare']=data_test['Fare']/data_test['FamilySize']





#fill in missing ages and fares.

median_fare = np.zeros((2,3))
median_ages = np.zeros((2,3))

for i in range(0,2):
	for j in range(0,3):
		median_fare[i,j] = data_test[(data_test['Gender'] == i ) & (data_test['Pclass'] ==j+1)]['Fare'].dropna().median()

for i in range(0,2):
	for j in range(0,3):
		data_test.loc[ (data_test.Fare.isnull() & (data_test['Gender']==i) & (data_test['Pclass']==j+1)), 'Fare']=median_fare[i,j]

for i in range(0,2):
	for j in range(0,3):
		median_ages[i,j] = data_test[(data_test['Gender'] == i ) & (data_test['Pclass'] ==j+1)]['Age'].dropna().median()



for i in range(0,2):
	for j in range(0,3):
		data_test.loc[ (data_test.Age.isnull() & (data_test['Gender']==i) & (data_test['Pclass']==j+1)), 'AgeFill']=median_ages[i,j]

        
data_test['Fare2']=2
data_test.loc[(data_test['Fare']<10),'Fare2']=1
data_test.loc[(data_test['Fare']>40),'Fare2']=3

#Normalize data for fare

from sklearn.preprocessing import scale
data_test['Fare']=scale(data_test['Fare'])


#Embarked
data_test['Embarked']=data_test['Embarked'].fillna("S")
data_test['Embarked'][data_test['Embarked']=="S"]=0
data_test['Embarked'][data_test['Embarked']=="C"]=1
data_test['Embarked'][data_test['Embarked']=="Q"]=2


#Ticket
data_test['Tix']=data_test['Ticket'].str.split().str.get(0)





print data["Title"].head(10)



0        Mr.
1       Mrs.
2      Miss.
3       Mrs.
4        Mr.
5        Mr.
6        Mr.
7    Master.
8       Mrs.
9       Mrs.
Name: Title, dtype: object


/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#encode labels for tix & title

from sklearn.preprocessing import LabelBinarizer
l_e1 = LabelBinarizer()
l_e2 = LabelEncoder()


#data["Tix"] = l_e1.fit_transform(data["Tix"])


#data_test["Tix"] = l_e1.transform(data_test["Tix"])

data["Title"] = l_e2.fit_transform(data["Title"])
data_test["Title"] = l_e2.transform(data_test["Title"])

from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()
data_cat = pandas.DataFrame(one_hot.fit_transform(data[['Title','Embarked']]).toarray())
data_test_cat = pandas.DataFrame(one_hot.transform(data_test[['Title','Embarked']]).toarray())

data.info()
data_test.info()
#print data_cat.shape()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
Gender         891 non-null int64
Title          891 non-null int64
Mother         891 non-null int64
FamilySize     891 non-null int64
Fare2          891 non-null int64
AgeFill        891 non-null float64
dtypes: float64(3), int64(10), object(5)
memory usage: 125.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 18 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object

/Users/diptikarmarkar/tensorflow/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


Drop unused columns for the model

In [8]:
#dropping variables from data

#data = data.drop(['Name', 'Age', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Agemissing'], axis = 1)
#data = data.drop(['PassengerId'], axis = 1)

#print data.describe()


data2 = data[["Pclass","AgeFill","Gender","Fare","Parch","SibSp"]]
data3 = np.hstack((data[["Gender","Pclass","Fare","FamilySize","Parch",'AgeFill']],data_cat))
target = data["Survived"]

#one hot encoder 

 




#dropping variables from data_test
#data_test = data_test.drop(['Name', 'Age', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Agemissing'], axis = 1)

#print data_test.describe()

list_to_write = data_test['PassengerId']
list_to_write = list_to_write.values 

#data_test = data_test.drop(['PassengerId'], axis = 1)

test_data2 = data_test[["Pclass","AgeFill","Gender","Fare","Parch","SibSp"]]
test_data3 = np.hstack((data_test[["Gender","Pclass","Fare","FamilySize","Parch","AgeFill"]],data_test_cat))
#print test_data3.info()
#print data3.info()

Creating separate validation and train datasets from the train data

1. Basic splitting
2. Through cross validation - SKIP
3. Lasso??

In [9]:
#plain splitting
train_data, validation_data, train_target, valid_target = train_test_split(data3, target, test_size = 0.15)


valid_data2=validation_data



In [10]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=2000, min_samples_split =25)

In [11]:
from sklearn import preprocessing
#scaler = preprocessing.StandardScaler()
#train_data = scaler.fit_transform(train_data)
#validation_data = scaler.transform(validation_data)
#test_data3 = scaler.transform(test_data3)

#validation_data = validation_data.values
#train_data=train_data.values

#test_data3 = test_data3.values

In [12]:

clf.fit(train_data, train_target)
pred = clf.predict(test_data3)
predori = clf.predict(validation_data)
predtrain = clf.predict(train_data)

df = pandas.DataFrame(validation_data)


#a = pandas.crosstab(predori, [valid_data2['Pclass'], valid_data2['Gender']])

#b = pandas.crosstab(valid_data2['Survived'], [valid_data2['Pclass'], valid_data2['Gender']])

#print a 
#print b

In [13]:
#print accuracy and confusion matrix and save test predictions is csv file


final = np.concatenate(([list_to_write], [pred]), axis = 0)


#print final

#print list_to_write
from sklearn.metrics import accuracy_score
acc = accuracy_score(predori, valid_target)
print "Validation Accuracy: ", acc

acc2 = accuracy_score(predtrain, train_target)
print "Train accuracy: ", acc2

from sklearn.metrics import classification_report
print(classification_report(valid_target, predori, labels=None,target_names=None,))
#header = "PassengerId, Survived"
#np.set_printoptions(suppress = True)
np.savetxt("mydata.csv", np.transpose(final), delimiter= ",", fmt = '%.1i',comments='',header = "PassengerId,Survived")



Validation Accuracy:  0.84328358209
Train accuracy:  0.86393659181
             precision    recall  f1-score   support

          0       0.89      0.86      0.88        86
          1       0.76      0.81      0.79        48

avg / total       0.85      0.84      0.84       134



In [14]:
#print model scores for each variable

print clf.feature_importances_

[ 0.18659247  0.10830486  0.15883272  0.06243939  0.01702729  0.09236447
  0.01472819  0.04593415  0.21120889  0.05976604  0.00721543  0.01501366
  0.01178164  0.00879081]
